# ETEO for Order Execution
This tutorial is to demonstrate an example of using ETEO to do order execution

## Step1: Import Packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
from pathlib import Path
import os
import torch

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.transition.builder import build_transition
from collections import Counter

## Step2: Import Configs

In [2]:
parser = argparse.ArgumentParser(description='Download Alpaca Datasets')
parser.add_argument("--config", default=osp.join(ROOT, "configs", "order_execution",
                                                 "order_execution_BTC_eteo_eteo_adam_mse.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")
parser.add_argument("--test_style", type=str, default="-1")
args,_ = parser.parse_known_args()

cfg = Config.fromfile(args.config)
task_name = args.task_name
cfg = replace_cfg_vals(cfg)

## Step3: Build Dataset

In [4]:
dataset = build_dataset(cfg)

The 18_th segment length is less than the min length so it won't be tested


## Step4: Build Environment

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="train"))
valid_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="valid"))
test_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="test"))

In [6]:
train_environment.df.head()

,system_time,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,...,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
0,2021-04-07 11:33:41.122161+00:00,55896.285,0.01,4.448599e+06,3.899402e+06,-8.945138e-08,-0.000065,-0.000073,-0.000154,-0.000162,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1176.72998,0.0
1,2021-04-07 11:34:41.122161+00:00,55948.685,1.43,1.243244e+06,3.606502e+06,-1.277957e-05,-0.000013,-0.000242,-0.000242,-0.000259,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
2,2021-04-07 11:35:41.122161+00:00,56013.785,0.01,3.069094e+06,1.575759e+06,-8.926374e-08,-0.000009,-0.000070,-0.000088,-0.000173,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
3,2021-04-07 11:36:41.122161+00:00,55903.575,7.17,1.220819e+06,1.321487e+06,-6.412827e-05,-0.000182,-0.000188,-0.000225,-0.000310,...,0.0,312.040009,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
4,2021-04-07 11:37:41.122161+00:00,55899.995,0.01,2.011287e+06,3.083005e+06,-8.944544e-08,-0.000129,-0.000130,-0.000143,-0.000201,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0


In [7]:
valid_environment.df.head()

,system_time,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,...,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
0,2021-04-13 10:44:00.423670+00:00,62541.695,9.15,5.787658e+05,106771.290598,-7.315120e-05,-7.331109e-05,-0.000073,-0.000074,-0.000139,...,0.0,0.0,0.0,0.0,233.169998,0.0,0.0,0.0,0.0,0.000000
1,2021-04-13 10:45:00.423670+00:00,62533.385,0.01,3.625578e+05,933043.632293,-7.995729e-08,-2.398719e-07,-0.000054,-0.000097,-0.000097,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
2,2021-04-13 10:46:00.423670+00:00,62600.005,0.01,1.293376e+06,162076.891581,-7.987220e-08,-1.465655e-04,-0.000148,-0.000159,-0.000199,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,2021-04-13 10:47:00.423670+00:00,62555.790,0.02,6.849401e+05,606836.562291,-1.598573e-07,-4.747762e-05,-0.000052,-0.000057,-0.000108,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,191.479996
4,2021-04-13 10:48:00.423670+00:00,62555.785,0.01,2.627973e+05,806676.814266,-7.992866e-08,-1.078238e-04,-0.000138,-0.000206,-0.000298,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [8]:
test_environment.df.head()

,system_time,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,...,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
0,2021-04-17 00:38:00.281751+00:00,61357.035,0.01,297283.360397,131197.795034,-8.149025e-08,-0.000015,-0.000109,-0.000135,-0.000138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-04-17 00:39:00.281751+00:00,61399.575,0.01,580371.169864,422586.530538,-8.143379e-08,-0.000044,-0.000052,-0.000056,-0.000122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-04-17 00:40:00.281751+00:00,61319.665,0.05,70460.909745,128334.135492,-4.076996e-07,-0.000023,-0.000027,-0.000062,-0.000062,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-04-17 00:41:00.281751+00:00,61396.515,0.01,464370.628504,431485.596255,-8.143785e-08,-0.000225,-0.000241,-0.000249,-0.000253,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-04-17 00:42:00.281751+00:00,61331.785,0.01,215821.000056,47201.539144,-8.152379e-08,-0.000007,-0.000041,-0.000058,-0.000076,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step5: Build Net

In [9]:
action_dim = train_environment.action_dim
state_dim = train_environment.state_dim

cfg.act.update(dict(action_dim=action_dim, state_dim=state_dim))
cfg.cri.update(dict(action_dim=action_dim, state_dim=state_dim))

act = build_net(cfg.act)
cri = build_net(cfg.cri)

## Step6: Build Optimizer

In [10]:
act_optimizer = build_optimizer(cfg, default_args=dict(params=act.parameters()))
cri_optimizer = None

## Step7: Build Loss

In [11]:
criterion = build_loss(cfg)

## Step8: Build Transition

In [12]:
transition = build_transition(cfg)

## Step9: Build Agent

In [13]:
agent = build_agent(cfg, default_args=dict(action_dim=action_dim,state_dim=state_dim,act=act,cri=cri,act_optimizer=act_optimizer,cri_optimizer=cri_optimizer,criterion=criterion,transition=transition,device=device))

## Step10: Build Trainer

In [14]:
trainer = build_trainer(cfg, default_args=dict(train_environment=train_environment,valid_environment=valid_environment,test_environment=test_environment,agent=agent,device=device))

| Arguments Keep work_dir: E:\workspace\RA\TradeMaster\work_dir/order_execution_BTC_eteo_eteo_adam_mse


## Step11: Train, Valid and Test

In [15]:
trainer.train_and_valid()

Train Episode: [1/10]
+--------------+--------------+-----------------+
|  Cash Left   |     TWAP     | Cash Left Ratio |
+--------------+--------------+-----------------+
| 48950.975103 | 37461.153500 |    30.671297%   |
+--------------+--------------+-----------------+
Valid Episode: [1/10]
+--------------+--------------+-----------------+
|  Cash Left   |     TWAP     | Cash Left Ratio |
+--------------+--------------+-----------------+
| 36578.324606 | 38610.495450 |    -5.263260%   |
+--------------+--------------+-----------------+
Train Episode: [2/10]
+--------------+--------------+-----------------+
|  Cash Left   |     TWAP     | Cash Left Ratio |
+--------------+--------------+-----------------+
| 49264.609981 | 37461.153500 |    31.508524%   |
+--------------+--------------+-----------------+
Valid Episode: [2/10]
+--------------+--------------+-----------------+
|  Cash Left   |     TWAP     | Cash Left Ratio |
+--------------+--------------+-----------------+
| 36582.3453

In [16]:
trainer.test()


Resume checkpoint E:\workspace\RA\TradeMaster\work_dir/order_execution_BTC_eteo_eteo_adam_mse\checkpoints\best.pth
Test Best Episode
+--------------+--------------+-----------------+
|  Cash Left   |     TWAP     | Cash Left Ratio |
+--------------+--------------+-----------------+
| 36590.664350 | 38610.495450 |    -5.231301%   |
+--------------+--------------+-----------------+


0